# 目录 <a id='menu'></a>
[头文件](#header)

[获取Headers](#getHeaders)

[获取Cookies](#getCookies)

[生成代理ip池](#generateProxyIp)

[获取题库页面](#getPages)

[获取题目网址](#getQuestionUrls)

[下载保存题目](#loadQuestions)

[主函数](#main)

# 头文件 <a id='header'></a> [返回](#menu)

In [1]:
# 问他网：http://www.7wenta.com/
# 用户：13651544
# 密码：a200314
import codecs
import requests
import json
from bs4 import BeautifulSoup
import re
import time
import random
requests.adapters.DEFAULT_RETRIES = 3

# 获取Headers <a id='getHeaders'></a> [返回](#menu)

In [2]:
def GetUserAgent():
    user_agent_list=[
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]
    return random.choice(user_agent_list)

def GetHeaders():
    headers = {
        'User-Agent':GetUserAgent()
    }
    return headers

def GetTimeString():
    return str(int(time.time()))+str(random.randint(0,9))+str(random.randint(0,9))+str(random.randint(0,9))

# 获取cookies <a id='getCookies'></a> [返回](#menu)

In [3]:
def GetCookies(proxy):
    loginUrl = 'http://www.7wenta.com/login2.json?%s' % GetTimeString()
    headers = {
        'Accept':'*/*',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        #'Content-Length':64,
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.7wenta.com',
        'Origin':'http://www.7wenta.com',
        'Referer':'http://www.7wenta.com/',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.96 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    data = {
        'account':'13651544',
        'pwd':'678127483bdd304fada4018689663cbe',
        'auto':'false'
    }
    try:
        response = requests.post(loginUrl,data=data,headers=headers,proxies=proxy)
        if response.cookies == '<RequestsCookieJar[]>':
            print('获取cookies为空。。')
            return None
        print('获取cookies成功！\n%s'%response.cookies)
        return response.cookies
    except:
        print('获取cookies失败。。')
        return None

    

# 生成代理ip池 <a id='generateProxyIp'></a> [返回](#menu)

In [4]:
def GenerateProxyList(oldProxyList):
    print('正在更新代理ip集。。')
    proxyList = []
    ipUrl = 'http://www.xicidaili.com/'
    session = requests.session()
    try:
        response = session.get(ipUrl,headers=GetHeaders())
        #print(response.content.decode('utf-8'))
        soup = BeautifulSoup(response.content.decode('utf-8'),'html.parser')
        trs = soup.find_all('tr',attrs={'class':re.compile(r'(odd)|()')})
        for tr in trs:
            #print(tr)
            tds = tr.find_all('td')
            if len(tds)>6 and tds[6].text!='1分钟':
                #print(tds[6])
                proxy = {'http':tds[1].string+':'+tds[2].string,
                         'https':tds[1].string+':'+tds[2].string}
                if proxy in oldProxyList:
                    continue
                ipTestUrl = 'http://ip.chinaz.com/getip.aspx'
                try:
                    response = session.get(ipTestUrl,proxies=proxy,timeout=5)
                    proxyList.append(proxy)
                    print('获取代理ip%s成功！'%proxy)
                except:
                    #print('代理ip%s失效。。'%proxy)
                    continue
            if(len(proxyList)==10):
                print('代理ip生成完成！')
                break
    except:
        print('获取代理ip失败。。')
    session.close
    return proxyList

def GetProxy(proxyList):
    return random.choice(proxyList)

# 获取题库页面 <a id='getPages'></a> [返回](#menu)

In [5]:
def GetPages(nPage):
    beginPage = 'http://www.7wenta.com/tiku/g301.html'
    oldPages = set()
    newPages = set()
    newPages.add(beginPage)
    while (len(newPages)!=0) and (len(oldPages) < nPage):
        new_url = newPages.pop()
        oldPages.add(new_url)
        try:
            response = requests.get(new_url)
            soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
            urls = soup.find_all('a',href=re.compile(r'http://www.7wenta.com/tiku/'))
            for url in urls:
                if url is None:
                    continue
                if url not in newPages and url not in oldPages:
                    newPages.add(url['href'])
        except:
            print('获取页面%s失败。。'%new_url)
    print('共找到%d个页面！'%len(oldPages))
    return oldPages

# 获取题目网址 <a id='getQuestionUrls'></a> [返回](#menu)

In [6]:
def GetQuestionUrls(pages):
    questionUrls = set()
    for page in pages:
        try:
            response = requests.get(page)
            soup = BeautifulSoup(response.content.decode(),'html.parser')
            urls = soup.find_all('a',class_='ansBtn r')
            for url in urls:
                #print(url)
                questionUrl = url['href'].replace('html','json?%s&fromSearch=false'%GetTimeString()).replace('topic/','topic/answer/')
                questionUrls.add(questionUrl)
        except:
            print('打开页面%s失败。。'%page)
    print('共找到%d条题目！'%len(questionUrls))
    return questionUrls

def SaveQuestionUrls(questionUrls):
    with codecs.open('questionUrls.txt','a','utf-8') as wf:
        for item in questionUrls:
            wf.write(item + '\n')

# 下载保存题目 <a id='loadQuestions'></a> [返回](#menu)

In [10]:
def LoadQuestions(questionUrls, saveFile, oldProxyList, savedUrls):
    postHeaders = {
        'Accept':'*/*',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        #'Content-Length':64,
        'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8',
        'Host':'www.7wenta.com',
        'Origin':'http://www.7wenta.com',
        'Referer':'http://www.7wenta.com/',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.96 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
    }
    postData = {
        'account':'13651544',
        'pwd':'678127483bdd304fada4018689663cbe',
        'auto':'false'
    }
    
    getHeaders = {
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding':'gzip, deflate, sdch',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Cache-Control':'max-age=0',
        'Connection':'keep-alive',
        'Host':'www.7wenta.com',
        'Referer':'http://www.7wenta.com/',
        #'Upgrade-Insecure-Requests':1,
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
    }
    
    with codecs.open(saveFile,'a','utf-8') as wf:
        number = 1
        index = 0
        sess = requests.Session()
        proxyList = GenerateProxyList(oldProxyList)
        if len(proxyList) == 0:
            print('代理ip列表为空。。结束爬虫。。')
            return
        
        
        while(True):
            sess.headers = postHeaders
            sess.proxies = proxyList[index]
            index += 1
            if index >= len(proxyList):
                proxyList = GenerateProxyList(oldProxyList)
                if len(proxyList) == 0:
                    print('代理ip列表为空。。结束爬虫。。')
                    return
                index = 0
            time.sleep(random.uniform(3,8)) 
            try:
                sess.post('http://www.7wenta.com/login2.json?%s' % GetTimeString(), data=postData)
                print('登陆成功！')
                break
            except:
                print('登录失败。。')
        for url in questionUrls:
            if url.split('.')[2].split('/')[-1] in savedUrls:
                print('该题目已下载！')
                continue
            while(True):
                try:
                    sess.headers = getHeaders
                    response = sess.get(url)
                    js = json.loads(response.content.decode('utf-8'))
                    if 'value' in js and js['value'] != None:
                        wf.write(response.content.decode('utf-8'))
                        wf.write('\n')
                        print('%d、下载题目%s成功！'%(number,url))
                        with codecs.open('savedUrls.txt','a','utf-8') as su:
                            su.write(url.split('.')[2].split('/')[-1])
                            su.write('\n')
                        time.sleep(random.uniform(3,8))
                        break
                    else:
                        if 'resultCode' in js and js['resultCode']['code'] == -11:
                            print('需要填验证码。。')
                        elif 'resultCode' in js:
                            print('ip已失效，正在换新ip。。 %d, %s'%(js['resutlCode']['code'], js['resultCode']['detail']))
                        else:
                            print('ip已失效，正在换新ip。。')
                        while(True):
                            sess.headers = postHeaders
                            sess.proxies = proxyList[index]
                            index += 1
                            if index >= len(proxyList):
                                proxyList = GenerateProxyList(oldProxyList)
                                if len(proxyList) == 0:
                                    print('代理ip列表为空。。结束爬虫。。')
                                    return
                                index = 0
                            time.sleep(random.uniform(3,8)) 
                            try:
                                sess.post('http://www.7wenta.com/login2.json?%s' % GetTimeString(), data=postData)
                                print('登陆成功！')
                                break
                            except:
                                print('登录失败。。')
                except:
                    print('下载题目失败。。')
                    while(True):
                        sess.headers = postHeaders
                        sess.proxies = proxyList[index]
                        index += 1
                        if index >= len(proxyList):
                            proxyList = GenerateProxyList(oldProxyList)
                            if len(proxyList) == 0:
                                print('代理ip列表为空。。结束爬虫。。')
                                return
                            index = 0
                        time.sleep(random.uniform(3,8)) 
                        try:
                            sess.post('http://www.7wenta.com/login2.json?%s' % GetTimeString(), data=postData)
                            print('登陆成功！')
                            break
                        except:
                            print('登录失败。。')
            if number % 100 == 0:
                print('休息1分钟。')
                time.sleep(60)
            number += 1
    print('下载保存完成！')

# 主函数 <a id='main'></a> [返回](#menu)

In [8]:
pages = GetPages(100)
#pages = ['http://www.7wenta.com/tiku/g111.html']
questionUrls = GetQuestionUrls(pages)
#questionUrls = ['http://www.7wenta.com/topic/answer/9814DAEB6422AC27EE5E1B85950694C7.json?1497441966718&fromSearch=false']

共找到100个页面！
共找到965条题目！


In [9]:
SaveQuestionUrls(questionUrls)

In [8]:
questionUrls = set()
with codecs.open('questionUrls.txt','r','utf-8') as lf:
    for line in lf:
        #print(line.strip())
        questionUrls.add(line.strip())
print(len(questionUrls))

880


In [ ]:
oldProxyList = []
savedUrls = set()
with codecs.open('savedUrls.txt','r','utf-8') as lf:
    for line in lf:
        savedUrls.add(line.strip())
print(len(savedUrls))
LoadQuestions(questionUrls,'answer.txt',oldProxyList,savedUrls)

5034
正在更新代理ip集。。
获取代理ip{'http': '27.184.50.115:8118', 'https': '27.184.50.115:8118'}成功！
获取代理ip{'http': '58.245.100.110:8118', 'https': '58.245.100.110:8118'}成功！
获取代理ip{'http': '144.255.230.186:8118', 'https': '144.255.230.186:8118'}成功！
获取代理ip{'http': '122.72.32.88:80', 'https': '122.72.32.88:80'}成功！
获取代理ip{'http': '180.105.170.222:808', 'https': '180.105.170.222:808'}成功！
获取代理ip{'http': '182.242.179.85:8118', 'https': '182.242.179.85:8118'}成功！
获取代理ip{'http': '218.11.84.198:8118', 'https': '218.11.84.198:8118'}成功！
获取代理ip{'http': '113.200.214.164:9999', 'https': '113.200.214.164:9999'}成功！
获取代理ip{'http': '124.239.177.85:8080', 'https': '124.239.177.85:8080'}成功！
获取代理ip{'http': '112.115.43.80:9999', 'https': '112.115.43.80:9999'}成功！
代理ip生成完成！
登陆成功！
该题目已下载！
1、下载题目http://www.7wenta.com/topic/answer/3968AD9406FE5A629A35C7438D334A31.json?1502429139740&fromSearch=false成功！
该题目已下载！
需要填验证码。。
登陆成功！
2、下载题目http://www.7wenta.com/topic/answer/1A537BC424BF667EF690C815D5AEB08B.json?1502429129769&fromSearch

In [3]:
answer = set()
i = 0
with codecs.open('answer.txt','r','utf-8') as lf:
    for line in lf:
        answer.add(line.strip())
        i += 1
print('题目数：',len(answer))
print('行数：',i)

题目数： 3734
行数： 3743


# 数据去重

In [15]:
import json
import codecs
answerId = set()
with codecs.open('answer.txt','r','utf-8') as lf:
    for line in lf:
        js = json.loads(line)
        value = js['value']
        if value['id'] in answerId:
            print(value['id'])
        else:
            answerId.add(value['id'])

# 保存地址去重

In [20]:
import codecs
answerId = set()
with codecs.open('savedUrls.txt','r','utf-8') as lf:
    for line in lf:
        if line.strip() not in answerId:
            answerId.add(line.strip())
        else:
             print(line.strip())

In [19]:
import codecs
answerId = set()
with codecs.open('savedUrls.txt','r','utf-8') as lf:
    for line in lf:
        answerId.add(line.strip())
with codecs.open('savedUrls.txt','w','utf-8') as wf:
    for line in answerId:
        wf.write(line + '\n')